In [1]:
import numpy as np

LDA:
1. Calculate the assumed common covariance matrix
2. Calculate the means for each of the two classes
3. Plug into the formula $f(x)=sign(w^Tx+b)$
    1. $w=\Sigma^{-1}(\mu_1-\mu_{-1})$
    2. $b=\frac{1}{2}(\mu_{-1}+\mu_1)^T\Sigma^{-1}(\mu_{-1}-\mu_1)+log\frac{N_1}{N_{-1}}$
    
QDA:
1. Calculate the covariance matrix for each of the two classes
2. Calculate the means for each of the two classes
3. Plug into the formula $f(x)=sign(w^Tx+b)$
    1. $w=\Sigma^{-1}(\mu_1-\mu_{-1})$
    2. $b=\frac{1}{2}(\mu_{-1}+\mu_1)^T\Sigma^{-1}(\mu_{-1}-\mu_1)+log\frac{N_1}{N_{-1}}$
    

In [98]:
class LDA:
    def __init__(self):
        self.w=0
        self.b=0

    def fit(self,X,y):
        classes= np.unique(y)
        covariance_matrix = np.cov(X[y==classes[0]].T)
        mean_0=np.mean(X[y==classes[0]], axis=0)
        print(mean_0)
        mean_1=np.mean(X[y==classes[1]], axis=0)

        covariance_matrix_inverse = np.linalg.inv(covariance_matrix)
        no_0 = (y==classes[0]).sum()
        no_1=(y==classes[1]).sum()

        self.w=covariance_matrix_inverse.dot(mean_1-mean_0)
        print((mean_0+mean_1).dot(covariance_matrix_inverse))
        self.b=(1/2)*(mean_0+mean_1).T.dot(covariance_matrix_inverse).dot((mean_0-mean_1))+np.log(no_1/no_0)

        
    def predict_proba(self, Xtest):
        f = self.w.T.dot(Xtest)+self.b
        print("Predicted: ",f)
        return f
    def predict(self, Xtest):
        f = np.sign(self.w.T.dot(Xtest)+self.b)
        print("Predicted: ",f)
        return f
    def get_params(self):
        print("w: ",self.w ,"\nb: ",self.b)
        return (self.w, self.b)
    
class QDA:
    def fit(X,y):
        return NotImplementedError
    def predict_proba(Xtest):
        return NotImplementedError
    def predict(Xtest):
        return NotImplementedError
    def get_params():
        return NotImplementedError
    

class NB:
    def fit(X,y):
        return NotImplementedError
    def predict_proba(Xtest):
        return NotImplementedError
    def predict(Xtest):
        return NotImplementedError
    def get_params():
        return NotImplementedError
    



In [104]:
lda=LDA()
X = np.random.normal(3, 2.5, size=(10, 4))
Y = np.random.randint(2,size=10)
print(Y)
lda.fit(X,Y)
lda.get_params()

[1 1 1 0 1 0 0 1 0 1]
[2.67624602 4.56589187 2.72409731 1.00858302]
[-1.98363422e+17 -3.03894164e+16 -5.02266932e+16 -1.42770689e+16]
w:  [-2.57433837e+16 -3.94390458e+15 -6.51836424e+15 -1.85286209e+15] 
b:  1.224154321497406e+17


(array([-2.57433837e+16, -3.94390458e+15, -6.51836424e+15, -1.85286209e+15]),
 1.224154321497406e+17)

In [122]:


lda.predict(X[0])

Predicted:  -1.0


-1.0